In [ ]:
"""
調整傾斜臉的主程式
"""

from collections import OrderedDict
import numpy as np
import cv2
from os import listdir, path

from os.path import join, isfile, isdir

FACIAL_LANDMARKS_IDXS = OrderedDict([("mouth", (48, 68)),
                                     ("right_eyebrow", (17, 22)),
                                     ("left_eyebrow", (22, 27)),
                                     ("right_eye", (36, 42)),
                                     ("left_eye", (42, 48)),
                                     ("nose", (27, 36)),
                                     ("jaw", (0, 17))
                                     ])

def shape_to_np(shape, dtype="int"):
    coordinates_list = np.zeros((68, 2), dtype=dtype)
    for i in range(0, 68):
        coordinates_list[i] = (shape.part(i).x, shape.part(i).y)
    return coordinates_list

class FaceAligner:
    def __init__(self, predictor, desiredLeftEye=(0.35, 0.35), desiredFaceWidth=256, desiredFaceHeight=None):

        self.predictor = predictor
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight

        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth

    def align(self, image, gray, rect):

        shape = self.predictor(gray, rect)
        shape = shape_to_np(shape)

        (lStart, lEnd) = FACIAL_LANDMARKS_IDXS["left_eye"]
        (rStart, rEnd) = FACIAL_LANDMARKS_IDXS["right_eye"]
        leftEyePts = shape[lStart:lEnd]
        rightEyePts = shape[rStart:rEnd]

        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180

        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist

        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2, (leftEyeCenter[1] + rightEyeCenter[1]) // 2)

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale) # - angle

        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

        return output, angle


import cv2
import dlib
# import glob

predictor_path = "/content/drive/MyDrive/basic_emotion/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
fa = FaceAligner(predictor, desiredFaceWidth=256)

def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)

    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

def file_name_list(filepath):
     files = listdir(filepath)
     file_list = []
     for f in files:
         fullpath = join(filepath, f)
         if isdir(fullpath):
             # filename, file_extension = path.splitext(f)
             file_list.append(f)
     return file_list
